<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_106/back_to_basics/langchain_basics_4_langserve-in-progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# serving with LangServe

Let's bundle up the application we just wrote, and serve it via API.
Note, this code will work if:

1. you run it on your local machine
2. you have the `langserve` package installed on top of the other requirements
3. you have a remote instance that allows you to get shell access



In [8]:
# install LangServe with:
%pip install "langserve[all]"

Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install fastapi uvicorn

Note: you may need to restart the kernel to use updated packages.


We need to bundle up the application we just wrote inside of a file named "serve.py" and then we can serve it via API.

This file contains three things
1. our code that we wrote in the previous step
2. Our FastAPI appliction
3. A route definition from which we serve the chain

In [10]:
#!/usr/bin/env python

from typing import List
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain.tools import DuckDuckGoSearchRun
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage
from langserve import add_routes

# Load Retriever
loader = WebBaseLoader("https://www.austinchronicle.com/events")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# Create our tools
retriever_tool = create_retriever_tool(
    retriever,
    "austin_events_search",
    "Search for information about events in Austin. For any questions about austin events, you may use this tool",
)

search = DuckDuckGoSearchRun()
tools = [retriever_tool, search]


# Create our agent
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# App definition
app = FastAPI(
    title="LangChain Server",
    version="1.0",
    description="A simple API for our LangChain Quickstart",
)

# Add our chain routes
class Input(BaseModel):
    input: str
    chat_history: List[BaseMessage] = Field(
    ...,
    extra={"widget": {"type": "chat", "input": "location"}},
    )

class Output(BaseModel):
    output: str

add_routes(
    app,
    agent_executor.with_types(input_type=Input, output_type=Output),
    path="/agent",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

NameError: name 'FastAPI' is not defined

Let's execute the file and see what happens

In [11]:
python langchain_basics_5_langserve.py

SyntaxError: invalid syntax (2937407649.py, line 1)

You should see something like this:

'''
(.venv) (base) Colins-MacBook-Pro:back_to_basics colinmcnamara$ python langchain_basics_5_langserve.py 
INFO:     Started server process [56341]
INFO:     Waiting for application startup.

 __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
|  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
|  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
|  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
|  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
|_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|

LANGSERVE: Playground for chain "/agent/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /agent/playground/
LANGSERVE:
LANGSERVE: See all available routes at /docs/

LANGSERVE: ⚠️ Using pydantic 2.6.4. OpenAPI docs for invoke, batch, stream, stream_log endpoints will not be generated. API endpoints and playground should work as expected. If you need to see the docs, you can downgrade to pydantic 1. For example, `pip install pydantic==1.10.13`. See https://github.com/tiangolo/fastapi/issues/10360 for details.

INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
'''